# Training Notebook

In [ ]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from ml_attack import get_no_mod, LWEDataset, get_filename_from_params
from ml_attack.utils import get_continuous_reduction_default_params, get_default_params, get_percentage_true_b, get_train_default_params, cmod

import numpy as np

from scipy.linalg import circulant
from ml_attack.lwe import neg_circ

from ml_attack.continuous_reduction import ContinuousReduction
from concurrent.futures import ProcessPoolExecutor

from itertools import product

np.set_printoptions(linewidth=np.inf)

## Dataset creation

Training debug:

In [2]:
params = get_default_params()
params.update(get_continuous_reduction_default_params())
params.update(get_train_default_params())
params.update({
    'n': 128,
    'q': 3329,
    'secret_type': 'binary',

    'num_gen': 4,
    'seed': 0,

    'num_matrices': 7,
    'reduction_max_size': 400,
    'float_type': 'd',
    'matrix_config': 'original',
    'reduction_factor': None,
    'reduction_resampling': True,
    'warmup_steps': 1,
    'bkz_block_sizes': "20:45:5",
    
    'penalty': 4,
    'verbose': True,

    "train_percentages": [0.1, 0.25, 1],
})

filename = get_filename_from_params(params)

reload = True
if os.path.exists(filename) and reload:
    print(f"Loading dataset from {filename}")
    dataset = LWEDataset.load_reduced(filename)
    params = dataset.params
else:
    print(f"Generating dataset and saving to {filename}")
    dataset = LWEDataset(params)
    dataset.initialize()
    dataset.attack(
        stop_strategy="hour",
        stop_after=1
    )

Generating dataset and saving to ./data_n_128_k_1_s_binary_d74b6.pkl
Attacking 7 matrices using 8 threads.
- Algo: bkz2.0_20 | Updated 198/198 | Mean std_B: 3896.30
- Algo: bkz2.0_20 | Updated 198/198 | Mean std_B: 3907.79
- Algo: bkz2.0_20 | Updated 198/198 | Mean std_B: 3937.26
- Algo: bkz2.0_20 | Updated 198/198 | Mean std_B: 3932.59
- Algo: bkz2.0_20 | Updated 229/229 | Mean std_B: 3272.94
- Algo: bkz2.0_20 | Updated 198/198 | Mean std_B: 3939.11
- Algo: bkz2.0_20 | Updated 214/214 | Mean std_B: 3677.96
Tour 1 | Time: 53.22s | Mean std_B: 3782.26
[BEST 10% STD] True B is the best candidate: 58 / 143 (40.56%)
[BEST 10% STD] Expected true B is best candidate: 40.51%


Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


✔️ Patched scikit-learn (once).
[BEST 25% STD] True B is the best candidate: 143 / 358 (39.94%)
[BEST 25% STD] Expected true B is best candidate: 38.55%
True B is the best candidate: 506 / 1433 (35.31%)
Expected true B is best candidate: 34.51%
b'infinite loop in babai'
Upgrading float type to ld
b'infinite loop in babai'
Upgrading float type to ld
b'infinite loop in babai'
Upgrading float type to ld
b'infinite loop in babai'
Upgrading float type to ld
b'infinite loop in babai'
Upgrading float type to ld
b'infinite loop in babai'
Upgrading float type to ld
b'infinite loop in babai'
Upgrading float type to ld
- Algo: bkz2.0_20 | Updated 235/235 | Mean std_B: 3012.77
- Algo: bkz2.0_20 | Updated 241/241 | Mean std_B: 2768.69
- Algo: bkz2.0_20 | Updated 233/233 | Mean std_B: 3055.92
- Algo: bkz2.0_20 | Updated 246/246 | Mean std_B: 2665.86
- Algo: bkz2.0_20 | Updated 246/246 | Mean std_B: 2623.69
- Algo: bkz2.0_20 | Updated 245/245 | Mean std_B: 2591.44
- Algo: bkz2.0_20 | Updated 249/249 

KeyboardInterrupt: 

In [3]:
get_percentage_true_b(dataset, verbose=True)

True B is the best candidate: 1308 / 1309 (99.92%)


np.float64(0.9992360580595875)

In [4]:
num_gen = dataset.params['num_gen']
n = dataset.params['n']
k = dataset.params['k']
q = dataset.params['q']

In [5]:
dataset.A.shape

(256, 64)

In [6]:
dataset.A

array([[-1570.,   152.,   401., ...,   669.,  1408.,  1553.],
       [-1553., -1570.,   152., ...,    41.,   669.,  1408.],
       [-1408., -1553., -1570., ...,  1428.,    41.,   669.],
       ...,
       [  462.,   814.,   151., ...,   341.,  -916., -1631.],
       [ 1631.,   462.,   814., ..., -1604.,   341.,  -916.],
       [  916.,  1631.,   462., ...,    66., -1604.,   341.]])

In [17]:
import numpy as np
from scipy.linalg import qr

def extract_invertible_submatrix(R):
    # Ensure R is 2n x n
    rows, cols = R.shape
    
    # QR decomposition with row pivoting (on R^T to pivot rows of R)
    Q, R_qr, row_pivots = qr(R.T, pivoting=True)

    # Get the indices of the first n pivoted rows
    selected_rows = row_pivots[:cols]

    # Extract the square submatrix
    R_sub = R[selected_rows, :]

    # Check if the submatrix is invertible
    if np.linalg.matrix_rank(R_sub) == cols:
        return R_sub, selected_rows
    else:
        raise ValueError("Could not find invertible submatrix.")

In [9]:
dataset.R[0][0]

array([ 3., -1., -6., 11.,  8.,  2.,  6., 18., -6.,  4., -4., -4., -3., -1.,  7.,  6.])

In [8]:
neg_circ(dataset.R[0][0]).T

array([[  3.,  -1.,  -6.,  11.,   8.,   2.,   6.,  18.,  -6.,   4.,  -4.,  -4.,  -3.,  -1.,   7.,   6.],
       [ -6.,   3.,  -1.,  -6.,  11.,   8.,   2.,   6.,  18.,  -6.,   4.,  -4.,  -4.,  -3.,  -1.,   7.],
       [ -7.,  -6.,   3.,  -1.,  -6.,  11.,   8.,   2.,   6.,  18.,  -6.,   4.,  -4.,  -4.,  -3.,  -1.],
       [  1.,  -7.,  -6.,   3.,  -1.,  -6.,  11.,   8.,   2.,   6.,  18.,  -6.,   4.,  -4.,  -4.,  -3.],
       [  3.,   1.,  -7.,  -6.,   3.,  -1.,  -6.,  11.,   8.,   2.,   6.,  18.,  -6.,   4.,  -4.,  -4.],
       [  4.,   3.,   1.,  -7.,  -6.,   3.,  -1.,  -6.,  11.,   8.,   2.,   6.,  18.,  -6.,   4.,  -4.],
       [  4.,   4.,   3.,   1.,  -7.,  -6.,   3.,  -1.,  -6.,  11.,   8.,   2.,   6.,  18.,  -6.,   4.],
       [ -4.,   4.,   4.,   3.,   1.,  -7.,  -6.,   3.,  -1.,  -6.,  11.,   8.,   2.,   6.,  18.,  -6.],
       [  6.,  -4.,   4.,   4.,   3.,   1.,  -7.,  -6.,   3.,  -1.,  -6.,  11.,   8.,   2.,   6.,  18.],
       [-18.,   6.,  -4.,   4.,   4.,   3.,   1.,  -7.,

In [123]:
cmod(neg_circ(dataset.R[0][0]).T @ dataset.A, q)

array([[-15., -51., -41., -22., -21., -62., -34.,  49., -14.,  11., -43.,  -4.,  52.,  49.,  17., -35.],
       [ 35., -15., -51., -41., -22., -21., -62., -34.,  49., -14.,  11., -43.,  -4.,  52.,  49.,  17.],
       [-17.,  35., -15., -51., -41., -22., -21., -62., -34.,  49., -14.,  11., -43.,  -4.,  52.,  49.],
       [-49., -17.,  35., -15., -51., -41., -22., -21., -62., -34.,  49., -14.,  11., -43.,  -4.,  52.],
       [-52., -49., -17.,  35., -15., -51., -41., -22., -21., -62., -34.,  49., -14.,  11., -43.,  -4.],
       [  4., -52., -49., -17.,  35., -15., -51., -41., -22., -21., -62., -34.,  49., -14.,  11., -43.],
       [ 43.,   4., -52., -49., -17.,  35., -15., -51., -41., -22., -21., -62., -34.,  49., -14.,  11.],
       [-11.,  43.,   4., -52., -49., -17.,  35., -15., -51., -41., -22., -21., -62., -34.,  49., -14.],
       [ 14., -11.,  43.,   4., -52., -49., -17.,  35., -15., -51., -41., -22., -21., -62., -34.,  49.],
       [-49.,  14., -11.,  43.,   4., -52., -49., -17.,

In [115]:
cmod(dataset.R[0] @ dataset.A, q)

array([[ -15.,  -51.,  -41.,  -22.,  -21.,  -62.,  -34.,   49.,  -14.,   11.,  -43.,   -4.,   52.,   49.,   17.,  -35.],
       [ -28.,  -60.,  -16.,   21.,   -4.,  119.,  -10.,  -28.,  -31.,   -9.,  -17.,   34.,   18.,  -42.,   19.,   69.],
       [   1.,  -75.,  -22.,   18.,   43.,  -15.,  -66.,   18.,  -21.,    3.,  -35.,  -39.,   63.,  -75.,  -15.,    2.],
       [  21.,   62.,   34.,  -49.,   14.,  -11.,   43.,    4.,  -52.,  -49.,  -17.,   35.,  -15.,  -51.,  -41.,  -22.],
       [  -4.,  119.,  -10.,  -28.,  -31.,   -9.,  -17.,   34.,   18.,  -42.,   19.,   69.,   28.,   60.,   16.,  -21.],
       [ -26.,  -29.,   66.,   20.,   -2.,  -95.,   16.,   -3.,    1.,    7.,   14.,   -1.,  -59.,   54.,  -92.,   15.],
       [  46.,  -23.,  -28.,  -50.,   39.,  -30.,   48.,   89.,  -56.,   -3.,  -35.,  -48.,   20.,   40.,   24.,  -64.],
       [ -24.,   74.,  -41.,   59.,  -42.,   53.,   32.,    0.,   28.,   24.,  -61.,  -26.,    5.,   21.,   71.,  -41.],
       [ -18.,   21.,   -3.,   3

## Recurrent Reduction (not working)

In [70]:
A = dataset.A.reshape((num_gen, k*n, k*n))
A_blocks = A.reshape(num_gen, k, n, k, n).transpose(0, 1, 3, 2, 4)
# Shape: (batches, k, k, n, n)
l = 2  # provided divisor of n
A_rlwe_split = A_blocks.reshape(num_gen, k, k, l, n//l, l, n//l).transpose(0, 1, 2, 3, 5, 4, 6)
# Shape: (batches, k, k, l, l, n//l, n//l)
first_row_blocks = A_rlwe_split[:, :, :, 0, :, :, :]  # Take l matrices from the 0th row
# Shape: (batches, k, k, l, n//l, n//l)
# Reshape to combine all (num_gen, k, k, l, n//l, n//l) blocks into a single array of matrices (total_blocks, n//l, n//l)
all_blocks = first_row_blocks.reshape(-1, n // l, n // l)
all_blocks

array([[[-1116.,   914., -1098.,   943.,  -983.,   655.,  -399.,   360.,   -22.,  1322.,    32.,   828.,  1325.,  1233.,  -861.,   386.],
        [ -938., -1116.,   914., -1098.,   943.,  -983.,   655.,  -399.,   360.,   -22.,  1322.,    32.,   828.,  1325.,  1233.,  -861.],
        [  297.,  -938., -1116.,   914., -1098.,   943.,  -983.,   655.,  -399.,   360.,   -22.,  1322.,    32.,   828.,  1325.,  1233.],
        [  204.,   297.,  -938., -1116.,   914., -1098.,   943.,  -983.,   655.,  -399.,   360.,   -22.,  1322.,    32.,   828.,  1325.],
        [ -180.,   204.,   297.,  -938., -1116.,   914., -1098.,   943.,  -983.,   655.,  -399.,   360.,   -22.,  1322.,    32.,   828.],
        [ 1631.,  -180.,   204.,   297.,  -938., -1116.,   914., -1098.,   943.,  -983.,   655.,  -399.,   360.,   -22.,  1322.,    32.],
        [  123.,  1631.,  -180.,   204.,   297.,  -938., -1116.,   914., -1098.,   943.,  -983.,   655.,  -399.,   360.,   -22.,  1322.],
        [ 1287.,   123.,  1631.,  

In [71]:
num_matrices = all_blocks.shape[0]
print(f"Total number of matrices to reduce: {num_matrices}")
args = []
for i in range(num_matrices):
    reduction = ContinuousReduction(params)
    args.append((reduction.to_state_dict(), all_blocks[i], 1))

def continuous_reduction_worker(args):
    reduction_state, matrix_to_reduce, times = args
    reduction = ContinuousReduction.from_state_dict(reduction_state)
    R, matrix_to_reduce = reduction.reduce(matrix_to_reduce, times=times)
    return R, [reduction.to_state_dict(), matrix_to_reduce, times]

n_jobs = 4  # Adjust based on your system's capabilities
with ProcessPoolExecutor(max_workers=n_jobs) as executor:
    R_reduced = []
    for i, (R, result) in enumerate(executor.map(continuous_reduction_worker, args)):
        R_reduced.append(R)
        args[i] = result

R_reduced = np.stack(R_reduced)

Total number of matrices to reduce: 2
- Running Flatter with alpha 0.001...- Running Flatter with alpha 0.001...

- Saving initial best stds (Mean std_B 86.79703479930672)
- Saving initial best stds (Mean std_B 85.8463167011379)


In [73]:
all_blocks[0]

array([[-1116.,   914., -1098.,   943.,  -983.,   655.,  -399.,   360.,   -22.,  1322.,    32.,   828.,  1325.,  1233.,  -861.,   386.],
       [ -938., -1116.,   914., -1098.,   943.,  -983.,   655.,  -399.,   360.,   -22.,  1322.,    32.,   828.,  1325.,  1233.,  -861.],
       [  297.,  -938., -1116.,   914., -1098.,   943.,  -983.,   655.,  -399.,   360.,   -22.,  1322.,    32.,   828.,  1325.,  1233.],
       [  204.,   297.,  -938., -1116.,   914., -1098.,   943.,  -983.,   655.,  -399.,   360.,   -22.,  1322.,    32.,   828.,  1325.],
       [ -180.,   204.,   297.,  -938., -1116.,   914., -1098.,   943.,  -983.,   655.,  -399.,   360.,   -22.,  1322.,    32.,   828.],
       [ 1631.,  -180.,   204.,   297.,  -938., -1116.,   914., -1098.,   943.,  -983.,   655.,  -399.,   360.,   -22.,  1322.,    32.],
       [  123.,  1631.,  -180.,   204.,   297.,  -938., -1116.,   914., -1098.,   943.,  -983.,   655.,  -399.,   360.,   -22.,  1322.],
       [ 1287.,   123.,  1631.,  -180.,  

In [81]:
RA = neg_circ(R_reduced[0][0]).T @ all_blocks[0] % q
RA[RA > q // 2] -= q
RA

array([[  -30.,    19.,   -21.,   -35.,    15.,   -36.,    36.,   -18.,   -70.,     4.,    -7.,   -42.,   -18.,    29.,    31.,    21.],
       [  975., -1180., -1464.,  -473.,  -860.,  1382.,  -900.,   179.,  1484.,   921.,  -926., -1445.,  -254.,  1347.,    84.,    14.],
       [-1140.,  1550.,  1226., -1238.,  1604.,   121., -1515.,   693.,  -572.,  -676.,  1386.,  -207., -1339.,   728.,  -345., -1572.],
       [  698.,   696.,   -54., -1144., -1513.,   950.,  -167.,   752.,    84.,   868.,  -986.,  -203.,   832.,  -884., -1473.,   759.],
       [ -426.,  1041.,   817., -1465.,  1635.,   508.,   374.,  1038.,  -466.,  -365.,   248.,  -835., -1454., -1587.,  1372.,   735.],
       [ -453.,   -83.,  1162.,  -594.,  1314.,   327.,   -68.,  1579.,  -180.,  -915.,  -985.,   399.,  1243.,  -544.,   669.,   251.],
       [-1370., -1371.,   719.,  -982.,  1208.,  1641.,   471.,   463.,   219.,  -900.,  -760.,  1474.,  1544.,  -649., -1108.,   117.],
       [  -53.,  -906.,  -130.,   664.,  

In [38]:
num_rows = 2 * (n // l)
block_size = n // l
R_reshaped = R_reduced.reshape((num_gen, k, k, l, num_rows, block_size))  # [b, k, k, l, 2n/l, n/l]
R_reshaped

array([[[[[[ -6.,  13.,   7.,   7.],
           [ 10.,  -4.,   0.,   0.],
           [  3.,   9.,   7.,  10.],
           [ -3.,   8.,  16., -18.],
           [  1.,  -6.,  -4.,   1.],
           [ 18.,  16.,   0.,   5.],
           [ -6.,   2., -17., -15.],
           [  0., -20.,   8.,  -5.]],

          [[  2.,   6.,  16.,   1.],
           [ 16.,   2.,  -8.,   2.],
           [-10., -11.,   3.,  13.],
           [ -4.,  16., -11.,   2.],
           [-14.,   8., -17.,   6.],
           [ -4.,  -2., -13.,  16.],
           [ 17.,   0.,  -9.,  16.],
           [  2.,  14.,  12.,   7.]]]]]])

In [43]:
for b in range(num_gen):
  # Precompute neg_circ matrices for all (i,j,li,row_idx)
  negcirc_dict = {}
  for i in range(k):
      for j in range(k):
          for li in range(l):
              for r in range(num_rows):
                  vec = R_reshaped[b, i, j, li, r]
                  negcirc_dict[(i, j, li, r)] = neg_circ(vec).T

choosen_matrix_0 = negcirc_dict[(0, 0, 0, 0)]
choosen_matrix_1 = negcirc_dict[(0, 0, 1, 0)]

In [44]:
choosen_matrix_0, choosen_matrix_1

(array([[ -6.,  13.,   7.,   7.],
        [ -7.,  -6.,  13.,   7.],
        [ -7.,  -7.,  -6.,  13.],
        [-13.,  -7.,  -7.,  -6.]]),
 array([[  2.,   6.,  16.,   1.],
        [ -1.,   2.,   6.,  16.],
        [-16.,  -1.,   2.,   6.],
        [ -6., -16.,  -1.,   2.]]))

In [ ]:
def build_twisted_matrix(blocks):
    """
    Given a list of l matrices (R1', R2', ..., Rl'), construct the twisted RLWE matrix.
    """
    l = len(blocks)
    block_size = blocks[0].shape[0]
    full_size = l * block_size
    result = np.zeros((full_size, full_size))

    for i in range(l):
        for j in range(l):
            idx = (j - i) % l
            sign = (-1) ** ((j - i) < 0)
            result[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size] = sign * blocks[idx]
    return result

In [47]:
twisted_block = build_twisted_matrix([choosen_matrix_0, choosen_matrix_1])
twisted_block

array([[ -6.,  13.,   7.,   7.,   2.,   6.,  16.,   1.],
       [ -7.,  -6.,  13.,   7.,  -1.,   2.,   6.,  16.],
       [ -7.,  -7.,  -6.,  13., -16.,  -1.,   2.,   6.],
       [-13.,  -7.,  -7.,  -6.,  -6., -16.,  -1.,   2.],
       [ -2.,  -6., -16.,  -1.,  -6.,  13.,   7.,   7.],
       [  1.,  -2.,  -6., -16.,  -7.,  -6.,  13.,   7.],
       [ 16.,   1.,  -2.,  -6.,  -7.,  -7.,  -6.,  13.],
       [  6.,  16.,   1.,  -2., -13.,  -7.,  -7.,  -6.]])

In [51]:
twisted_block

array([[ -6.,  13.,   7.,   7.,   2.,   6.,  16.,   1.],
       [ -7.,  -6.,  13.,   7.,  -1.,   2.,   6.,  16.],
       [ -7.,  -7.,  -6.,  13., -16.,  -1.,   2.,   6.],
       [-13.,  -7.,  -7.,  -6.,  -6., -16.,  -1.,   2.],
       [ -2.,  -6., -16.,  -1.,  -6.,  13.,   7.,   7.],
       [  1.,  -2.,  -6., -16.,  -7.,  -6.,  13.,   7.],
       [ 16.,   1.,  -2.,  -6.,  -7.,  -7.,  -6.,  13.],
       [  6.,  16.,   1.,  -2., -13.,  -7.,  -7.,  -6.]])

In [30]:
def sample_unique_indices(base, length, num_samples, seed=None):
    """
    Samples unique integers in [0, base**length) and returns base-B representations.
    """
    rng = np.random.default_rng(seed)
    total = base ** length

    if num_samples > total:
        raise ValueError("Requested more unique samples than the total possible combinations.")

    sampled = rng.choice(total, size=num_samples, replace=False)
    # Convert each to base `base` representation of length `length`
    def to_base(x):
        digits = []
        for _ in range(length):
            digits.append(x % base)
            x //= base
        return digits[::-1]  # Most significant first

    return [to_base(idx) for idx in sampled]

def sample_full_R_matrices_no_repeat(R_raw, batches, k, l, n, num_samples, seed=None):
    """
    Samples num_samples unique R matrices (shape k*n, k*n) per batch without repetition.
    """
    rng = np.random.default_rng(seed)
    num_rows = 2 * (n // l)
    block_size = n // l
    R_reshaped = R_raw.reshape((batches, k, k, l, num_rows, block_size))  # [b, k, k, l, 2n/l, n/l]

    base = num_rows ** l               # choices per (i,j)
    length = k * k                     # number of blocks per matrix
    total_combinations = base ** length

    all_batched_R = []

    for b in range(batches):
        # Precompute neg_circ matrices for all (i,j,li,row_idx)
        negcirc_dict = {}
        for i in range(k):
            for j in range(k):
                for li in range(l):
                    for r in range(num_rows):
                        vec = R_reshaped[b, i, j, li, r]
                        negcirc_dict[(i, j, li, r)] = neg_circ(vec).T

        sampled_index_vectors = sample_unique_indices(base, length, num_samples, seed + b if seed else None)

        sampled_Rs = []
        for idx_vec in sampled_index_vectors:
            full_R = np.zeros((k * n, k * n))
            for block_idx, (i, j) in enumerate(product(range(k), repeat=2)):
                flat_index = idx_vec[block_idx]

                # Decode flat_index → l row indices (base `num_rows`)
                row_choices = []
                x = flat_index
                for _ in range(l):
                    row_choices.append(x % num_rows)
                    x //= num_rows
                row_choices = row_choices[::-1]

                parts = [
                    negcirc_dict[(i, j, li, row_choices[li])]
                    for li in range(l)
                ]
                twisted_block = build_twisted_matrix(parts)
                full_R[i*n:(i+1)*n, j*n:(j+1)*n] = twisted_block

            sampled_Rs.append(full_R)

        all_batched_R.append(sampled_Rs)

    return np.stack(all_batched_R)


In [31]:
def count_total_R_matrices(n, l, k):
    """
    Calculates the total number of different full R matrices generated
    from R_raw with parameters:
    
    n: polynomial degree
    l: number of RLWE splits per LWE block
    k: module rank

    Returns: total number of (k*n, k*n) R matrices
    """
    num_rows_per_block = 2 * (n // l)
    num_combinations_per_block = num_rows_per_block ** l
    total_blocks = k * k  # each full matrix has k^2 (i,j) blocks
    total_combinations = num_combinations_per_block ** total_blocks
    return total_combinations

total_R_matrices = count_total_R_matrices(n, l, k)
print(f"Total number of different full R matrices: {total_R_matrices}")

Total number of different full R matrices: 64


In [32]:
# Generate all twisted R matrices
total_R_matrices = 1
twisted_matrices = sample_full_R_matrices_no_repeat(R_reduced, num_gen, k, l, n, total_R_matrices, seed=42)

In [33]:
twisted_matrices

array([[[[ -6.,  13.,   7.,   7.,  -4.,  -2., -13.,  16.],
         [ -7.,  -6.,  13.,   7., -16.,  -4.,  -2., -13.],
         [ -7.,  -7.,  -6.,  13.,  13., -16.,  -4.,  -2.],
         [-13.,  -7.,  -7.,  -6.,   2.,  13., -16.,  -4.],
         [  4.,   2.,  13., -16.,  -6.,  13.,   7.,   7.],
         [ 16.,   4.,   2.,  13.,  -7.,  -6.,  13.,   7.],
         [-13.,  16.,   4.,   2.,  -7.,  -7.,  -6.,  13.],
         [ -2., -13.,  16.,   4., -13.,  -7.,  -7.,  -6.]]]])

In [34]:
import numpy as np

def multiply_sampled_Rs_by_A_vectorized(A, sampled_Rs):
    """
    A: ndarray of shape (batches, kn, kn)
    sampled_Rs: list of [batches][num_samples] with each R ∈ (kn, kn)

    Returns:
        result: ndarray of shape (batches, num_samples, kn, kn)
    """
    sampled_Rs_array = np.array(sampled_Rs)  # (batches, num_samples, kn, kn)

    # We want to do: result[b, s] = sampled_Rs[b, s] @ A[b]
    # A needs to be broadcasted to (batches, num_samples, kn, kn)
    batches, num_samples, kn, _ = sampled_Rs_array.shape
    A_expanded = A[:, None, :, :]  # (batches, 1, kn, kn)

    # Broadcasted matrix multiplication
    result = sampled_Rs_array @ A_expanded  # shape: (batches, num_samples, kn, kn)

    return result


In [35]:
RA = multiply_sampled_Rs_by_A_vectorized(A, twisted_matrices)
RA %= q
RA[RA > q // 2] -= q

print(f"Shape of RA: {RA.shape}")  # Should be (batches * num_samples, kn, kn)

Shape of RA: (1, 1, 8, 8)


In [36]:
RA

array([[[[   49.,    96.,  -140.,     2.,   558., -1464.,   349.,   368.],
         [-1655.,  1043.,   308.,  1461.,  1161., -1584.,    76.,  1374.],
         [ -914., -1274., -1568.,  1640.,  -797., -1320.,   299.,   790.],
         [ 1551.,   688.,   903.,  -414.,   236.,  -636.,   308.,  -417.],
         [ -558.,  1464.,  -349.,  -368.,    49.,    96.,  -140.,     2.],
         [-1161.,  1584.,   -76., -1374., -1655.,  1043.,   308.,  1461.],
         [  797.,  1320.,  -299.,  -790.,  -914., -1274., -1568.,  1640.],
         [ -236.,   636.,  -308.,   417.,  1551.,   688.,   903.,  -414.]]]])

## Enhancing RLWE (not working for MLWE)

In [24]:
dataset.R.squeeze().shape

(2, 32, 16)

In [25]:
R_reshaped = dataset.R.squeeze().reshape(num_gen, k, 2*n, k, n).transpose(0, 1, 3, 2, 4).reshape(num_gen, k * k, 2*n, n).squeeze()
R_reshaped

array([[[[ 13.,  15.,   3., ...,  -7., -11., -11.],
         [  7.,   2.,   7., ...,   4.,  -7.,   1.],
         [  2.,   7., -16., ...,  -7.,   1.,  -7.],
         ...,
         [  1.,  12.,  10., ...,  -3.,   0.,   4.],
         [ -4.,   7.,  -1., ...,   7., -16.,   9.],
         [ -3., -11., -14., ...,  -6.,  -7.,  10.]],

        [[  5.,  -3.,  -1., ...,   2.,   1.,  10.],
         [  3.,  -3.,   3., ...,  -1., -10.,  -2.],
         [ -3.,   3.,  12., ..., -10.,  -2.,  -3.],
         ...,
         [  7.,   8.,  -7., ...,  15., -11.,   0.],
         [  1.,  10.,   2., ...,   3.,  12.,  11.],
         [ -2.,   0.,  -4., ...,  -2.,   3.,   3.]],

        [[  7.,  -8.,  -1., ...,   1.,   1.,   0.],
         [  5.,  -1.,  -6., ...,  10.,   7., -17.],
         [ -2.,   5.,   2., ...,  -4.,  -3.,  15.],
         ...,
         [  8.,  -2.,   1., ...,   6.,  -4.,   8.],
         [ 14.,   9.,  21., ...,   9.,   3.,   3.],
         [ 12.,  -9.,  -2., ...,  -6.,   2.,  11.]],

        [[  0., 

In [26]:
dataset.R.squeeze().shape

(2, 32, 16)

In [9]:
RA = cmod(dataset.R.squeeze() @ dataset.A.reshape(num_gen, k*n, k*n), q)
RA

array([[[ 18., -22.,  82., ..., -31., -11.,  29.],
        [ 46.,  18., -22., ..., -52., -31., -11.],
        [-47.,   7., -22., ...,  10., -31.,  35.],
        ...,
        [-46.,  88.,  17., ...,  -4.,  87.,  23.],
        [-10., -49., -14., ...,  -1.,  78.,  -9.],
        [ 72.,  16.,  35., ...,  86., -43.,  82.]],

       [[ 22., -33.,  -3., ...,  12.,  18., -27.],
        [ 12., -16., -28., ..., -11., -31., -17.],
        [ 16.,  17.,  -8., ...,  26., -97.,   9.],
        ...,
        [-23., -62.,  68., ...,  31.,  25., -20.],
        [ 41.,  43.,   0., ...,  67.,  61.,  10.],
        [ 59., -46., -21., ...,  14.,  14.,  14.]]])

In [5]:
B = dataset.B.reshape(num_gen, -1)

In [6]:
RB = cmod(dataset.R.squeeze() @ B[..., None], q)
RB

array([[[  70.],
        [ 291.],
        [-188.],
        [ -50.],
        [-106.],
        [  42.],
        [ -36.],
        [-251.],
        [ -56.],
        [ 120.],
        [ 344.],
        [ 158.],
        [ -78.],
        [-200.],
        [ 193.],
        [  11.],
        [  83.],
        [-103.],
        [-149.],
        [  32.],
        [-178.],
        [ 189.],
        [ -73.],
        [ 118.],
        [  12.],
        [  89.],
        [ 147.],
        [-130.],
        [ -82.],
        [ 157.],
        [  57.],
        [ 262.]],

       [[  74.],
        [ 134.],
        [ -30.],
        [-161.],
        [  62.],
        [   2.],
        [ -98.],
        [ 116.],
        [-215.],
        [ 188.],
        [  -5.],
        [-105.],
        [ 204.],
        [ 158.],
        [ 138.],
        [-273.],
        [ -42.],
        [  27.],
        [-296.],
        [-233.],
        [  91.],
        [ 169.],
        [-139.],
        [  70.],
        [ -58.],
        [ -97.],
        [  1

In [10]:
RA[0, 0]

array([ 18., -22.,  82., -50.,   9.,  62.,  38., -46., 100., -15., -12.,  44., -52., -31., -11.,  29.])

In [ ]:
RA_s = dataset.R[0, 0] @ dataset.A[:, 0] % q
RA_s 

np.float64(3.0)

In [ ]:
rolled = np.roll(dataset.R[0, 0], shift=2)
rolled[0] = -rolled[0]
rolled[1] = -rolled[1]
RA_s = rolled @ dataset.A[:, 2] % q
RA_s

np.float64(3.0)

In [ ]:
R = circulant(dataset.R[0, 1])
tri = np.triu_indices(n, 1)
R[tri] *= -1
R

array([[  7.,  -1.,  12.,  -2.,  -1., -13.,  -5.,  -3.,  -5.,   9.,   9.,
         -7.,   1.,  -6.,   7.,   3.],
       [  3.,   7.,  -1.,  12.,  -2.,  -1., -13.,  -5.,   3.,  -5.,   9.,
          9.,  -7.,   1.,  -6.,   7.],
       [  7.,   3.,   7.,  -1.,  12.,  -2.,  -1., -13.,   5.,   3.,  -5.,
          9.,   9.,  -7.,   1.,  -6.],
       [ -6.,   7.,   3.,   7.,  -1.,  12.,  -2.,  -1.,  13.,   5.,   3.,
         -5.,   9.,   9.,  -7.,   1.],
       [  1.,  -6.,   7.,   3.,   7.,  -1.,  12.,  -2.,   1.,  13.,   5.,
          3.,  -5.,   9.,   9.,  -7.],
       [ -7.,   1.,  -6.,   7.,   3.,   7.,  -1.,  12.,   2.,   1.,  13.,
          5.,   3.,  -5.,   9.,   9.],
       [  9.,  -7.,   1.,  -6.,   7.,   3.,   7.,  -1., -12.,   2.,   1.,
         13.,   5.,   3.,  -5.,   9.],
       [  9.,   9.,  -7.,   1.,  -6.,   7.,   3.,   7.,   1., -12.,   2.,
          1.,  13.,   5.,   3.,  -5.],
       [ -5.,   9.,   9.,  -7.,   1.,  -6.,   7.,   3.,   7.,   1., -12.,
          2.,   1.,  1

In [ ]:
RA = R.T @ dataset.A % q
RA[RA > q // 2] -= q
RA

array([[ 1534.,   -18.,   248.,  -134.,   699.,   607.,  1578.,  -907.],
       [  913.,   303.,    99.,   523.,  -217.,   325.,  1575.,   596.],
       [   -6.,  -168.,  -859., -1476.,   755.,  1097.,   737.,  1389.],
       [ -716.,  -970., -1621.,  1655.,  -506.,   431., -1596.,   846.]])

In [ ]:
RB = R.T @ dataset.B % q
RB[RB > q // 2] -= q
RB

array([ 28.,  25., -93., -22., -59.,  90.,  65.,  39.])

In [11]:
big_R = np.stack([neg_circ(row).T for reduced_matrix in dataset.R for row in reduced_matrix])
big_R.shape

(64, 16, 16)

In [ ]:
big_RA = big_R @ dataset.A % q
big_RA[big_RA > q // 2] -= q
big_RA

array([[[  3.,   9.,  54., ..., -72.,   2., -14.],
        [ 14.,   3.,   9., ..., -54., -72.,   2.],
        [ -2.,  14.,   3., ..., -10., -54., -72.],
        ...,
        [ 10.,  54.,  72., ...,   3.,   9.,  54.],
        [-54.,  10.,  54., ...,  14.,   3.,   9.],
        [ -9., -54.,  10., ...,  -2.,  14.,   3.]],

       [[ 37., -40., -77., ...,  97.,  73.,  12.],
        [-12.,  37., -40., ...,  31.,  97.,  73.],
        [-73., -12.,  37., ...,   7.,  31.,  97.],
        ...,
        [ -7., -31., -97., ...,  37., -40., -77.],
        [ 77.,  -7., -31., ..., -12.,  37., -40.],
        [ 40.,  77.,  -7., ..., -73., -12.,  37.]],

       [[ 52., -68.,  17., ...,  -6., -22., -75.],
        [ 75.,  52., -68., ..., -30.,  -6., -22.],
        [ 22.,  75.,  52., ...,  74., -30.,  -6.],
        ...,
        [-74.,  30.,   6., ...,  52., -68.,  17.],
        [-17., -74.,  30., ...,  75.,  52., -68.],
        [ 68., -17., -74., ...,  22.,  75.,  52.]],

       ...,

       [[  2.,  40., -31

In [ ]:
np.std(big_RA, axis=-1)

array([[36.34126442, 37.15087482, 37.05654463, 36.19651226, 28.94283158,
        26.64934333, 35.35799061, 36.12132334],
       [52.84411036, 53.74476719, 55.53996309, 48.11184885, 42.66951488,
        41.14000486, 52.58980414, 55.05168027],
       [49.64561914, 48.83646179, 47.20434302, 46.63689527, 42.90978909,
        49.60846702, 50.06683034, 45.90411202],
       [55.05168027, 52.84411036, 53.74476719, 55.53996309, 48.11184885,
        42.66951488, 41.14000486, 52.58980414],
       [41.63231918, 49.61791511, 49.88925235, 48.93298989, 48.55151903,
        40.05543034, 47.08768416, 41.63231918],
       [36.12132334, 36.34126442, 37.15087482, 37.05654463, 36.19651226,
        28.94283158, 26.64934333, 35.35799061],
       [35.73776014, 36.01648928, 37.72515739, 37.61648575, 41.03885354,
        41.06930119, 32.78242669, 36.28618883],
       [59.75457618, 60.59999484, 57.59218154, 58.55753047, 58.06233181,
        60.40992779, 55.03848086, 60.1860397 ],
       [26.66429026, 26.66429026

In [ ]:
big_RB = big_R @ dataset.B % q
big_RB[big_RB > q // 2] -= q
big_RB

array([[  28.,   25.,  -93.,  -22.,  -59.,   90.,   65.,   39.],
       [ -89.,   26.,  152.,    2.,  -86., -192.,  -82.,   46.],
       [ -87., -101.,   57.,   66.,   47.,  -20.,  -63.,   -4.],
       [ -46.,  -89.,   26.,  152.,    2.,  -86., -192.,  -82.],
       [ -39.,  -69.,  -37.,  124.,  -64.,  104.,  -24.,   64.],
       [ -39.,   28.,   25.,  -93.,  -22.,  -59.,   90.,   65.],
       [-125.,   12.,  -10.,  -57.,  -21., -166.,   87.,   61.],
       [-141.,  146.,    1.,   31., -123.,  -76.,   94.,   55.],
       [   4.,  -89.,   28.,   23.,   36.,  -56.,  -89.,  -13.],
       [ -17.,   85.,   61.,  -51.,  -59.,   20.,  145.,  102.],
       [  24.,  -64.,  -39.,  -69.,  -37.,  124.,  -64.,  104.],
       [  20.,   63.,    4.,  -87., -101.,   57.,   66.,   47.],
       [ -47.,   20.,   63.,    4.,  -87., -101.,   57.,   66.],
       [  51.,   59.,  -20., -145., -102.,  -17.,   85.,   61.],
       [ -65.,    1.,  -23.,   34.,  -24.,   26.,  -32.,   64.],
       [-181.,   35.,  -3

In [ ]:
get_no_mod(params, big_RA, dataset.secret, big_RB)

array([[  28.,   25.,  -93.,  -22.,  -59.,   90.,   65.,   39.],
       [ -89.,   26.,  152.,    2.,  -86., -192.,  -82.,   46.],
       [ -87., -101.,   57.,   66.,   47.,  -20.,  -63.,   -4.],
       [ -46.,  -89.,   26.,  152.,    2.,  -86., -192.,  -82.],
       [ -39.,  -69.,  -37.,  124.,  -64.,  104.,  -24.,   64.],
       [ -39.,   28.,   25.,  -93.,  -22.,  -59.,   90.,   65.],
       [-125.,   12.,  -10.,  -57.,  -21., -166.,   87.,   61.],
       [-141.,  146.,    1.,   31., -123.,  -76.,   94.,   55.],
       [   4.,  -89.,   28.,   23.,   36.,  -56.,  -89.,  -13.],
       [ -17.,   85.,   61.,  -51.,  -59.,   20.,  145.,  102.],
       [  24.,  -64.,  -39.,  -69.,  -37.,  124.,  -64.,  104.],
       [  20.,   63.,    4.,  -87., -101.,   57.,   66.,   47.],
       [ -47.,   20.,   63.,    4.,  -87., -101.,   57.,   66.],
       [  51.,   59.,  -20., -145., -102.,  -17.,   85.,   61.],
       [ -65.,    1.,  -23.,   34.,  -24.,   26.,  -32.,   64.],
       [-181.,   35.,  -3

In [ ]:
A_s = big_RA @ dataset.secret
e = (big_RB - A_s) % q
e[e > q // 2] -= q
e

array([[-12.,  14., -24.,  18., -47., -36.,   2.,  26.],
       [-24.,  -7.,  18., -17., -20., -18., -11.,  16.],
       [-29., -11.,  11.,  21., -49., -43., -25.,  18.],
       [-16., -24.,  -7.,  18., -17., -20., -18., -11.],
       [ -3.,  -1.,  -1.,  42., -15.,  15.,  13.,  -9.],
       [-26., -12.,  14., -24.,  18., -47., -36.,   2.],
       [-27.,   4.,  -7.,   0., -25., -69.,   6.,  55.],
       [ 17.,   1.,  26., -13.,  -1.,  32.,  28.,  17.],
       [ 15., -57.,  21., -14.,   8.,  -7., -60.,   6.],
       [ -9.,  42.,  36.,  -7.,   6.,  -1.,  67.,  54.],
       [-13.,   9.,  -3.,  -1.,  -1.,  42., -15.,  15.],
       [ 43.,  25., -18., -29., -11.,  11.,  21., -49.],
       [ 49.,  43.,  25., -18., -29., -11.,  11.,  21.],
       [  7.,  -6.,   1., -67., -54.,  -9.,  42.,  36.],
       [ -3.,   4., -39., -43., -58.,  32.,  -3.,  43.],
       [-22.,  -2., -30., -21.,  10.,  25., -16.,   1.]])

In [ ]:
print("RA shape:", RA.shape)
print("dataset.A shape:", dataset.A.shape)
print("dataset.R shape:", dataset.R.shape)
print("dataset.secret shape:", dataset.secret.shape)

RA shape: (8, 8)
dataset.A shape: (8, 8)
dataset.R shape: (1, 16, 8)
dataset.secret shape: (8,)
